In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import pickle

In [4]:
# Stabilized Random Forest
import NeuralNetwork
from importlib import reload
reload(NeuralNetwork)
from NeuralNetwork import StaGen,BiomassDataset
import torch
from torch.utils.data import DataLoader
from torch.nn import MSELoss,init
from torch.optim import Adam,AdamW

ens_data=pd.read_csv('data/StaGen_dataset.csv')
ens_train,ens_test=train_test_split(ens_data,test_size=0.2)
x_train_stb=ens_train.iloc[:,:7]
y_train_stb=ens_train.iloc[:,7:]

x_test_stb=ens_test.iloc[:,:7]
y_test_stb=ens_test.iloc[:,7:]

tensor_x_test_stb=torch.tensor(x_test_stb.values,dtype=torch.float32)
tensor_y_test_stb=torch.tensor(y_test_stb.values,dtype=torch.float32)

tensor_train_stb=torch.tensor(ens_train.values,dtype=torch.float32)


nn_model=StaGen(7)

n_epochs=2000
batch_size=64

dataset=BiomassDataset(tensor_train_stb)
dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=True,drop_last=False)

criterion=MSELoss()
optimizer=AdamW(nn_model.parameters(),lr=0.0002)

best_epoch=None
best_score=float('inf')
best_rmse=None
best_model=None

for epoch in range(n_epochs):
    # training    
    nn_model.train()
    for batch,batch_data in enumerate(dataloader):
        optimizer.zero_grad()
        feat,label=batch_data[:,:7],batch_data[:,7:]
        pred=nn_model(feat)
        loss=criterion(pred,label)
        
        loss.backward()
        optimizer.step()

In [5]:
StaGen_state_dict=nn_model.state_dict()
with open('trained_model/StaGen.pkl','wb') as f:
    pickle.dump(StaGen_state_dict,f)